First, the scaffolding for the code after.

In [1]:
n = 2
n2 = n**2

def listcast(lin):
    # cast vector to list
    res = []
    for x in lin:
        res.append(x[0])
    return res


def lincast(mat):
    # cast matrix to vector 
    res = Matrix(SR, n**2, 1)
    for i in range(n):
        for j in range(n):
            res[i*n + j] = mat[i,j]
    return res

def matcast(lin):
    # cast vector to matrix
    res = Matrix(SR, n, n)
    for i in range(n):
        for j in range(n):
            res[i,j] = lin[i*n + j][0]
    return res

def matricize(list, sz):
    # cast list to matrix
    res = Matrix(SR, sz, sz)
    for i in range(sz):
        for j in range(sz):
            res[i,j] = list[i*sz + j]
    return res

def mul(a, b):
    return a * b

def to_sing(ideal):
    res = []
    for eqn in ideal.gens():
        res.append(eqn)
    return res

Generating the necessary variables

In [2]:
vars_rota = var(','.join('r{}{}'.format(i, j) for i in range(1,n2+1) for j in range(1,n2+1)))
rota = matricize(vars_rota, n2)

vars_e = var(','.join('e{}{}'.format(i, j) for i in range(1, n+1) for j in range(1, n+1)))
vars_x = var(','.join('x{}'.format(i) for i in range(1, n2+1)))
vars_y = var(','.join('y{}'.format(i) for i in range(1, n2+1)))

x = matricize(vars_x, n)
y = matricize(vars_y, n)
vars_rota, rota

(
(r11, r12, r13, r14, r21, r22, r23, r24, r31, r32, r33, r34, r41, r42, r43, r44),

[r11 r12 r13 r14]
[r21 r22 r23 r24]
[r31 r32 r33 r34]
[r41 r42 r43 r44]
)

Generating the equations defining the Rota variety.

In [3]:
def ROp(mat):
    return matcast(rota.T * lincast(mat))


In [4]:
raw_equations = ROp(x) * ROp(y) - ROp(ROp(x)*y + x*ROp(y))
equations = []

# adding equations defined on a basis
for i in range(n2):
    for j in range(n2):
        subst_dict_x = dict([
            (vars_x[i], 0) for i in range(n2)
        ])
        subst_dict_y = dict([
            (vars_y[i], 0) for i in range(n2)
        ])
        subst_dict_x[vars_x[i]] = 1
        subst_dict_y[vars_y[j]] = 1
        eqs = raw_equations.substitute(subst_dict_x).substitute(subst_dict_y)
        
        equations += listcast(lincast(eqs))


In [5]:
# helper function, 
# take a system of eqns with vars_x and vars_y 
# and retain only the coeffs if x_i and y_j
def take_only(where, i,j):
    subst_dict_x = dict([
        (vars_x[i], 0) for i in range(n2)
    ])
    subst_dict_y = dict([
        (vars_y[i], 0) for i in range(n2)
    ])
    subst_dict_x[vars_x[i]] = 1
    subst_dict_y[vars_y[j]] = 1
    return (where).substitute(subst_dict_x).substitute(subst_dict_y)


In [6]:
R.<r11,r12,r13,r14,r21,r22,r23,r24,r31,r32,r33,r34,r41,r42,r43,r44> = QQ[vars_rota]
I = R.ideal(equations)
I
irrelevant_ideal = R.ideal(r11,r12,r13,r14,r21,r22,r23,r24,r31,r32,r33,r34,r41,r42,r43,r44)
vars_rota = [r11,r12,r13,r14,r21,r22,r23,r24,r31,r32,r33,r34,r41,r42,r43,r44]

In [7]:
vars_rota   

[r11,
 r12,
 r13,
 r14,
 r21,
 r22,
 r23,
 r24,
 r31,
 r32,
 r33,
 r34,
 r41,
 r42,
 r43,
 r44]

In [8]:
# these are the real equations, first to from Tang-Zhang-Sun
assert I.reduce(r12*r13 - r11*r11 - r12*r21 - r13*r31) == 0
assert I.reduce(r12*r14 - r11*r12 - r12*r22 - r13*r32) == 0

In [9]:
for eq in equations:
    print(eq)

-r11^2 + r12*r13 - r12*r21 - r13*r31
-r11*r12 + r12*r14 - r12*r22 - r13*r32
-r11*r13 + r13*r14 - r12*r23 - r13*r33
r12*r13 - 2*r11*r14 + r14^2 - r12*r24 - r13*r34
-(r11 + r22)*r21 + r12*r23 - r13*r41
-r12*r21 - (r11 + r22)*r22 + r11*r22 + r12*r24 - r13*r42
-(r11 + r22)*r23 + r14*r23 - r13*r43
-r14*r21 + r13*r22 - (r11 + r22)*r24 + r14*r24 - r13*r44
-r11*(r12 + r31) + r11*r31 - r14*r31 - r21*r32 + r12*r33
-(r12 + r31)*r12 + r11*r32 - r14*r32 - r22*r32 + r12*r34
-(r12 + r31)*r13 + r13*r31 - r23*r32
-(r12 + r31)*r14 + r13*r32 - r24*r32
-(r12 + r42)*r21 - r14*r41 + r12*r43
-(r12 + r42)*r22 - r12*r41 + r11*r42 - r14*r42 + r12*r44
-(r12 + r42)*r23
-(r12 + r42)*r24 - r14*r41 + r13*r42
-r11*(r13 + r21) + r11*r21 - r14*r21 + r13*r22 - r23*r31
-r12*(r13 + r21) + r12*r21 - r23*r32
-(r13 + r21)*r13 + r11*r23 - r14*r23 + r13*r24 - r23*r33
-(r13 + r21)*r14 + r12*r23 - r23*r34
-(r21 + r24)*r21 + r21^2 - r11*r23 + r22*r23 - r23*r41
-(r21 + r24)*r22 + r21*r22 - r12*r23 + r22*r24 - r23*r42
-r13*r23 - (r

Computing the decomposition of the variety

In [10]:
min_decomp = I.minimal_associated_primes()

In [11]:
C1, C2,C3, C4, A, B = min_decomp

In [12]:
_=singular.LIB("primdec.lib")


def is_absolute(comp):
    S = singular.ideal(comp).absPrimdecGTZ()
    singular.setring(S)
    return singular.execute("size(absolute_primes)") == '1'

print(is_absolute(C1))
print(is_absolute(C2))
print(is_absolute(C3))
print(is_absolute(C4))
print(is_absolute(A))
print(is_absolute(B))

True
True
True
True
True
True


In [13]:
dimension(C1),dimension(C2),dimension(C3),dimension(C4)

(2, 2, 2, 2)

In [14]:
dimension(A), dimension(B)

(3, 3)

The minimal components of the variety are the curves C1-4 and spaces A,B

Computing the intersections:

In [15]:
def get_intersections(pairs):
    to_inter = []
    for a, b in pairs:
        if (a != b):
            cur_inter = (a + b).radical()
            to_inter.append(cur_inter)
    res = to_inter[0]
    for inter in to_inter:
        res = res.intersection(inter)
    return res

intersection_locus = (get_intersections([(a,b) for a in min_decomp for b in min_decomp])).radical()

In [16]:
intersection_locus.is_prime()

True

In [17]:
dimension(intersection_locus)

2

In [18]:
get_intersections([(a, b) for a in min_decomp[0:4] for b in min_decomp[0:4]]) == irrelevant_ideal

True

In [19]:
get_intersections([a, A] for a in min_decomp[0:4]) == irrelevant_ideal

True

In [20]:
get_intersections([a, B] for a in min_decomp[0:4]) == irrelevant_ideal

True

In [21]:
get_intersections([(A, B)]) == intersection_locus

True

In [22]:
A

Ideal (-r42*r43 + r41*r44, -r42^2 + r32*r44, -r41*r42 + r32*r43, -r41*r42 + r31*r44, -r41^2 + r31*r43, -r32*r41 + r31*r42, r24*r42 + r44^2, r24*r41 + r43*r44, r24*r32 + r42*r44, r24*r31 + r42*r43, -r24*r43 + r23*r44, r23*r42 + r43*r44, r23*r41 + r43^2, r23*r32 + r42*r43, r23*r31 + r41*r43, r34 - r42, r33 - r41, r22 + r44, r21 + r43, r14 + r44, r13 + r43, r12 + r42, r11 + r41) of Multivariate Polynomial Ring in r11, r12, r13, r14, r21, r22, r23, r24, r31, r32, r33, r34, r41, r42, r43, r44 over Rational Field

Now export:

In [23]:
def desagify(objs):
    res = ""
    for obj in objs:
        res = res + str(obj) + ','
    return res

In [24]:
print(desagify(vars_rota))

r11,r12,r13,r14,r21,r22,r23,r24,r31,r32,r33,r34,r41,r42,r43,r44,


In [25]:
print(desagify(equations))

-r11^2 + r12*r13 - r12*r21 - r13*r31,-r11*r12 + r12*r14 - r12*r22 - r13*r32,-r11*r13 + r13*r14 - r12*r23 - r13*r33,r12*r13 - 2*r11*r14 + r14^2 - r12*r24 - r13*r34,-(r11 + r22)*r21 + r12*r23 - r13*r41,-r12*r21 - (r11 + r22)*r22 + r11*r22 + r12*r24 - r13*r42,-(r11 + r22)*r23 + r14*r23 - r13*r43,-r14*r21 + r13*r22 - (r11 + r22)*r24 + r14*r24 - r13*r44,-r11*(r12 + r31) + r11*r31 - r14*r31 - r21*r32 + r12*r33,-(r12 + r31)*r12 + r11*r32 - r14*r32 - r22*r32 + r12*r34,-(r12 + r31)*r13 + r13*r31 - r23*r32,-(r12 + r31)*r14 + r13*r32 - r24*r32,-(r12 + r42)*r21 - r14*r41 + r12*r43,-(r12 + r42)*r22 - r12*r41 + r11*r42 - r14*r42 + r12*r44,-(r12 + r42)*r23,-(r12 + r42)*r24 - r14*r41 + r13*r42,-r11*(r13 + r21) + r11*r21 - r14*r21 + r13*r22 - r23*r31,-r12*(r13 + r21) + r12*r21 - r23*r32,-(r13 + r21)*r13 + r11*r23 - r14*r23 + r13*r24 - r23*r33,-(r13 + r21)*r14 + r12*r23 - r23*r34,-(r21 + r24)*r21 + r21^2 - r11*r23 + r22*r23 - r23*r41,-(r21 + r24)*r22 + r21*r22 - r12*r23 + r22*r24 - r23*r42,-r13*r23 - (r

In [26]:
for min_comp in min_decomp:
    print(desagify(min_comp.gens()))

r24*r42 + r44^2,r43,r41,r34,r33 - r44,r32,r31 - r42,r23,r22 + r44,r21,r14,r13 - r24,r12,r11 + r44,
-r41^2 + r21*r42,r44,r43,r34 + r42,r33 + r41,r32,r31,r24,r23,r22 - r41,r14 + r41,r13 + r21,r12,r11,
r34*r43 + r44^2,r42,r41,r33 + r44,r32,r31,r24,r23,r22 - r44,r21 - r43,r14,r13,r12 - r34,r11 + r44,
-r41^2 + r31*r43,r44,r42,r34,r33 - r41,r32,r24 + r43,r23,r22 + r41,r21,r14 + r41,r13,r12 + r31,r11,
-r42*r43 + r41*r44,-r42^2 + r32*r44,-r41*r42 + r32*r43,-r41*r42 + r31*r44,-r41^2 + r31*r43,-r32*r41 + r31*r42,r24*r42 + r44^2,r24*r41 + r43*r44,r24*r32 + r42*r44,r24*r31 + r42*r43,-r24*r43 + r23*r44,r23*r42 + r43*r44,r23*r41 + r43^2,r23*r32 + r42*r43,r23*r31 + r41*r43,r34 - r42,r33 - r41,r22 + r44,r21 + r43,r14 + r44,r13 + r43,r12 + r42,r11 + r41,
-r42*r43 + r41*r44,r34*r43 + r44^2,r34*r41 + r42*r44,-r34*r42 + r32*r44,r32*r43 + r42*r44,r32*r41 + r42^2,-r43^2 + r23*r44,r23*r42 - r41*r43,r23*r34 + r43*r44,r23*r32 + r42*r43,-r41*r43 + r21*r44,-r23*r41 + r21*r43,-r41^2 + r21*r42,r21*r34 + r42*r43,r2

In [27]:
print(desagify(intersection_locus.gens()))

r41 + r44,r34 - r42,r33 + r44,r31 + r42,r24 - r43,r22 + r44,r21 + r43,r14 + r44,r13 + r43,r12 + r42,r11 - r44,r43^2 - r23*r44,r42*r43 + r44^2,r32*r43 + r42*r44,r42^2 - r32*r44,r23*r42 + r43*r44,r23*r32 - r44^2,


Pretty-print:

In [28]:
def trim(ideal):
    gens = sorted(ideal.gens(), key=lambda x: x.degree())
    lins = [gen for gen in gens if gen.degree() == 1]
    substs = dict({
        # (a,a) for a in vars_rota
    })
    
    occurs = dict()
    for gen in gens:
        for var_ in gen.variables():
            if occurs.get(var_) == None:
                occurs[var_] = 1
            else:
                occurs[var_] += 1

    for lin in lins:
        to_elim = None
        for v in lin.variables():
            if occurs[v] == 1:
                to_elim = v
        if to_elim is not None:
            coeff = lin.monomial_coefficient(to_elim)
            lin_ = lin / coeff - to_elim
            substs[to_elim] = lin_
            gens.remove(lin)
    return (rota.subs(substs), gens)




In [29]:
def subs(pair, dict_):
    rota_, gens = pair
    new_gens = []
    for gen in gens:
        new_gens.append(gen.subs(dict_))
    return rota_.subs(dict_), new_gens


In [30]:

def __ward_off__():
    a,b,c,d,e = var('a,b,c,d,e')


    # print(latex(subs(trim(C1), {r44:a, r24:b, r42:c})))
    # print()
    # print(latex(subs(trim(C2), {r41:a, r21:b, r42:c})))
    # print()
    # print(latex(subs(trim(C3), {r44:a, r34:b, r43:c})))
    # print()
    # print(latex(subs(trim(C4), {r41:a, r31:b, r43:c})))

    print((subs(trim(C1), {r44:a, r24:b, r42:c})))
    print()
    print((subs(trim(C2), {r41:a, r21:b, r42:c})))
    print()
    print((subs(trim(C3), {r44:a, r34:b, r43:c})))
    print()
    print((subs(trim(C4), {r41:a, r31:b, r43:c})))



__ward_off__()


([ a  0 -b  0]
[ 0  a  0  b]
[-c  0 -a  0]
[ 0  c  0  a], [a^2 + b*c])

([ 0  0  b  a]
[ b -a  0  0]
[ 0  0  a  c]
[ a  c  0  0], [-a^2 + b*c])

([ a -b  0  0]
[-c -a  0  0]
[ 0  0  a  b]
[ 0  0  c  a], [a^2 + b*c])

([ 0  b  0  a]
[ 0  a  0  c]
[ b  0 -a  0]
[ a  0  c  0], [-a^2 + b*c])


In [31]:
# subs(trim(intersection_locus), {r44:a, r43:b, r42:c, r32:d, r23:e})

And now lets find the orbits:

In [32]:
def find_component_of(substs):
    res = []
    possible_comps = min_decomp + [intersection_locus]
    comp_names = {
        0 : 'C1',
        1 : 'C2',
        2 : 'C3',
        3 : 'C4',
        4 : 'A',
        5 : 'B',
        6 : 'Sing',
    }

    for i in range(len(possible_comps)):
        comp = possible_comps[i]

        for eq in comp.gens():
            if not eq.subs(substs) == 0:
                break
        else:
            res.append(comp_names[i])
    return res

In [33]:
rota * matrix([_ for _ in vars_e]).T

[e11*r11 + e12*r12 + e21*r13 + e22*r14]
[e11*r21 + e12*r22 + e21*r23 + e22*r24]
[e11*r31 + e12*r32 + e21*r33 + e22*r34]
[e11*r41 + e12*r42 + e21*r43 + e22*r44]

In [34]:
def rota_dict_from_gubarev(e_dict):
    to_subst = dict(
        (var, 0) for var in vars_rota
    )

    for i in range(n2):
        for j in range(n2):
            ei = vars_e[i]
            ej = vars_e[j]
            temp = e_dict.get(ei, 0)
            coeff = 0
            if temp != 0:
                coeff = temp.coefficient(ej)
            to_subst[vars_rota[j + i*n2]] = coeff

    return to_subst


gubarev_orbits = [
    {e21 : e12}, # M1
    {e12 : e21}, # M1 Transpose
    {e21 : e11}, # M2
    {e12 : e11}, # M2 Transpose
    {e21 : e11, e22 : e12}, # M3
    {e12 : e11, e22 : e21}, # M3 Transpose
    {e21 : -e11, e11 : e12}, # M4
    {e12 : -e11, e11 : e21}  # M4 Transpose
]


In [35]:
orbit_components = [
    find_component_of(rota_dict_from_gubarev(orb)) for orb in gubarev_orbits
]

orbit_components

[['A', 'B', 'Sing'],
 ['A', 'B', 'Sing'],
 ['A'],
 ['B'],
 ['C1'],
 ['C3'],
 ['C4'],
 ['C2']]

We see that the orbits from https://doi.org/10.1007/s00009-018-1234-5 lie in components:

- M1 is in the singular locus
- M2 and M2.T lie on the smooth general part of the variety
- M3 and M4 and their transposes lie on the exceptional curves

In [36]:
transpose_dict = {
    r11 : r11,
    r12 : r13,
    r13 : r12,
    r14 : r14,
    r21 : r31,
    r22 : r33,
    r23 : r32,
    r24 : r34,
    r31 : r21,
    r32 : r23,
    r33 : r22,
    r34 : r24,
    r41 : r41,
    r42 : r43,
    r43 : r42,
    r44 : r44
}

print(list(map(lambda x : transpose_dict[x], [_ for _ in vars_rota])))

[r11, r13, r12, r14, r31, r33, r32, r34, r21, r23, r22, r24, r41, r43, r42, r44]


Looking at Macaulay, transposition transforms 
$A \leftrightarrow B$, 
$C_1 \leftrightarrow C_3$, 
$C_2 \leftrightarrow C_4$ 

A simple attempt to verify. Does not work currently

In [37]:
SL.<a,b,c,d> = QQ['a,b,c,d']
# a,b,c,d = vars('a,b,c,d')


def gubarev_dict_from_aut(operator, phi, invphi):
    phi = matcast(phi)
    invphi = matcast(invphi)
    aut_op = lincast(
            invphi * 
            matcast(
                operator.T * 
                lincast(
                    phi * matrix([[e11, e12], [e21, e22]]) * invphi
                )
            ) *
            phi
        )
    e_dict = dict([
        (eij, 0) for eij in vars_e
    ])
    def take(eij):
        _ = dict(e_dict)
        _[eij] = 1
        return _

    aut_dict = dict([
        (eij, (matrix([e11, e12, e21, e22]) * aut_op.subs(take(eij)))[0][0]) for eij in vars_e
    ])
    return aut_dict


phi, invphi = matrix([a,b,c,d]).T, matrix([d,-b,-c,a]).T

matcast(invphi) * matcast(phi)

[-b*c + a*d          0]
[         0 -b*c + a*d]

In [38]:
def find_component_of_SL(substs):
    res = []
    possible_comps = min_decomp + [intersection_locus]
    comp_names = {
        0 : 'C1',
        1 : 'C2',
        2 : 'C3',
        3 : 'C4',
        4 : 'A',
        5 : 'B',
        6 : 'Sing',
    }

    for i in range(len(possible_comps)):
        comp = possible_comps[i]

        temp = []
        for eq in comp.gens():
            temp.append(eq.subs(substs))
        res.append(SL.ideal(temp).radical())
    return res

In [55]:
def dict_of_orbit_from_orbit_repr(orbit_repr_as_gubarev):
    operator = rota.subs(rota_dict_from_gubarev(orbit_repr_as_gubarev))
    gub_dict = gubarev_dict_from_aut(operator, phi, invphi)
    return rota_dict_from_gubarev(gub_dict)

In [66]:
[find_component_of(dict_of_orbit_from_orbit_repr(orb)) for orb in gubarev_orbits]

[['A', 'B', 'Sing'],
 ['A', 'B', 'Sing'],
 ['A'],
 ['B'],
 ['C1'],
 ['C3'],
 ['C4'],
 ['C2']]